In [ ]:
#Imports

from collections import Counter

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import time
import tqdm
import json
import re
import pandas as pd

In [ ]:
# importing the module
import json
  
# Opening JSON file
f_train = open('json_datasets/train.json', 'r')
raw_train = json.load(f_train)
f_train.close()

f_test = open('json_datasets/test.json', 'r')
raw_test = json.load(f_test)
f_test.close()

In [ ]:
def removeQuotes(lst):
    ret = []
    for s in lst:
        if s != '``' and s != "''":
            ret.append(s)
    return ret

def camelCaseSplit(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    d = [m.group(0) for m in matches]
    new_d = []
    for token in d:
        token = token.replace('(', '')
        token = token.replace(')', '')
        token_split = token.split('_')
        for t in token_split:
            #new_d.append(t.lower())
            new_d.append(t)
    return new_d

In [ ]:
def g2tPreprocess(raw):
    df = []
    for item in raw:
        graph = 'g2t:'
        for relation in item['relations']:
            graph += ' <H> ' + ' '.join(removeQuotes(relation[0])) + ' <R> '
            graph += ' '.join(camelCaseSplit(relation[1])) + ' <T> '
            graph += ' '.join(removeQuotes(relation[2]))

        ents = [' '.join(removeQuotes(entity)) for entity in item['entities']]
        text = item['text']
        for i in range(len(ents)):
            text = text.replace('<ENT_'+str(i)+'>', ents[i])
        sample = [graph, text]
        df.append(sample)
    return pd.DataFrame(df, columns=['source_text', 'target_text'])

In [ ]:
train_df = g2tPreprocess(raw_train)
test_df = g2tPreprocess(raw_test)
print(test_df)

                                            source_text                                        target_text
0     g2t: <H> Abilene Regional Airport <R> city Ser...  Abilene , Texas is served by the Abilene Regio...
1     g2t: <H> Abilene Regional Airport <R> city Ser...  Abilene Regional Airport serves the city of Ab...
2     g2t: <H> Adolfo Suárez Madrid–Barajas Airport ...  Adolfo Suárez Madrid–Barajas Airport can be fo...
3     g2t: <H> Adolfo Suárez Madrid–Barajas Airport ...  Adolfo Suárez Madrid–Barajas Airport is locate...
4     g2t: <H> Adolfo Suárez Madrid–Barajas Airport ...  Adolfo Suárez Madrid–Barajas Airport is locate...
...                                                 ...                                                ...
4923  g2t: <H> Twilight ( band ) <R> genre <T> Black...  Aaron Turner plays Electric guitar and played ...
4924  g2t: <H> Twilight ( band ) <R> genre <T> Black...  Electric guitar Aaron Turner played with Black...
4925  g2t: <H> Uruguay <R> leader Nam

In [ ]:
pip install --upgrade simplet5

     |████████████████████████████████| 1.2 MB 4.2 MB/s 
     |████████████████████████████████| 2.8 MB 16.1 MB/s 
     |████████████████████████████████| 919 kB 71.4 MB/s 
     |████████████████████████████████| 829 kB 47.4 MB/s 
     |████████████████████████████████| 332 kB 67.2 MB/s 
     |████████████████████████████████| 596 kB 70.4 MB/s 
     |████████████████████████████████| 132 kB 79.5 MB/s 
     |████████████████████████████████| 3.3 MB 71.6 MB/s 
     |████████████████████████████████| 895 kB 59.4 MB/s 
     |████████████████████████████████| 61 kB 712 kB/s 
     |████████████████████████████████| 1.1 MB 58.0 MB/s 
     |████████████████████████████████| 271 kB 59.9 MB/s 
     |████████████████████████████████| 160 kB 70.5 MB/s 
     |████████████████████████████████| 192 kB 74.5 MB/s 
  Created wheel for simplet5: filename=simplet5-0.1.3-py3-none-any.whl size=6831 sha256=a96bad12cd75ffe45fac272985e8371d1c9b98400ce74ceb99d576ce02f3d30c
  Stored in directory: /root/.cache/pi

In [ ]:
from simplet5 import SimpleT5

# instantiate
model = SimpleT5()

# load (supports t5, mt5, byT5 models)
model.from_pretrained("t5","t5-base")

Global seed set to 42


Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [11]:
model.load_model("t5","./simplet5", use_gpu=True)

In [12]:
f_dev = open('json_datasets/dev.json', 'r')
raw_dev = json.load(f_dev)
f_dev.close()

In [13]:
# from pycocoevalcap.bleu.bleu import Bleu
#bleu = Bleu(4)

dev_df = g2tPreprocess(raw_dev)
# print(dev_df)

#hyp = [model.predict(dev_df['source_text'][i])[0] for i in range(len(dev_df))]

hyp = model.predict(dev_df['source_text'][0])
# ret = bleu.compute_score(dev_df['target_text'], hyp)
print(dev_df['source_text'][0])
print(hyp)
# print(dev_df['source_text'][50])
# print(model.predict(dev_df['source_text'][50]))

g2t: <H> Aarhus <R> leader Name <T> Jacob Bundsgaard
['Jacob Bundsgaard is the leader of Aarhus.']


In [14]:
print(dev_df['target_text'][0])

the leader of Aarhus is Jacob Bundsgaard .


In [15]:
pip install "git+https://github.com/salaniz/pycocoevalcap.git"

  Cloning https://github.com/salaniz/pycocoevalcap.git to /tmp/pip-req-build-0g2pptlc
  Running command git clone -q https://github.com/salaniz/pycocoevalcap.git /tmp/pip-req-build-0g2pptlc
  Created wheel for pycocoevalcap: filename=pycocoevalcap-1.2-py3-none-any.whl size=104312215 sha256=6a2455bbbf0c18d9078f4631054b9f29fd6c42179ed38ee89dabe77fb2ea5277
  Stored in directory: /tmp/pip-ephem-wheel-cache-8n19k4e1/wheels/6f/c9/51/e266f0496048c16686e133d8e33644d692931a356bfb372aae
Successfully built pycocoevalcap


In [21]:
from collections import defaultdict

ref = defaultdict(list)
ptr = 0
same = defaultdict(list)

for i in range(len(dev_df)):
    if i > 0 and dev_df['source_text'][i] != dev_df['source_text'][i-1]:
        ptr += 1
    same[ptr].append(dev_df['target_text'][i])
    ref[i] = same[ptr]
    

In [22]:
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.meteor.meteor import Meteor

bleu = Bleu(4)
meteor = Meteor()
rouge = Rouge()
cider = Cider()

# hyp = [model.predict(dev_df['source_text'][i])[0] for i in range(len(dev_df))]

hyp = dict(zip(range(len(dev_df)), [model.predict(dev_df['source_text'][i]) for i in range(len(dev_df))]))
# ref = dict(zip(range(len(dev_df)), [[dev_df['target_text'][i]] for i in range(len(dev_df))]))
ret = bleu.compute_score(ref, hyp)
print('BLEU INP {0:}'.format(len(hyp)))
print('BLEU 1-4 {0:}'.format(ret[0]))
print('METEOR {0:}'.format(meteor.compute_score(ref, hyp)[0]))
print('ROUGE_L {0:}'.format(rouge.compute_score(ref, hyp)[0]))
print('Cider {0:}'.format(cider.compute_score(ref, hyp)[0]))

{'testlen': 28884, 'reflen': 32048, 'guess': [28884, 27242, 25600, 23958], 'correct': [23139, 16850, 12274, 8976]}
ratio: 0.9012730903644252
BLEU INP 1642
BLEU 1-4 [0.7179826190315747, 0.6308858173839672, 0.5550832484336957, 0.4895345313878751]
METEOR 0.4671733466105585
ROUGE_L 0.6279615299414455
Cider 3.0902349437196
